In [18]:
%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train, get_squad_train_chars
from itertools import chain

%aimport QANet
%aimport HRED_v2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%%time
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/'

data = load_squad_train(squad_path, with_chars=True)
i_to_w, w_to_i, voc_size = get_squad_train_voc(squad_path)
i_to_c, c_to_i, alphabet_size = get_squad_train_chars(squad_path)
glove_embs = get_glove_train_embs(squad_path, glove_path)

def get_w(idx):
    return i_to_w[idx]

# Some answers get broken in the process of tokenization, because some answer words are not properly split.
def filter_broken_answers(data):
    return zip(*[d for d in zip(*data) if d[0][0]])

print 'Examples total:', len(data[0])

data = filter_broken_answers(data)

print 'Working examples:', len(data[0])

Examples total: 87599
Working examples: 86474
CPU times: user 58.3 s, sys: 1.21 s, total: 59.5 s
Wall time: 59.5 s


In [3]:
data = [map(list, data[0]), map(list, data[1])]

In [4]:
# Originally contexts are split into sentences, this reverses that.
for i in xrange(len(data[0])):
    data[0][i].append(list(chain(*data[0][i][1][1:])))
    data[0][i][1] = data[0][i][1][0]

In [5]:
trim = 300
data = zip(*[(d0[:2] + [d0[2][:trim]], [d1[0], d1[1][:trim]]) for d0, d1 in zip(*data) if max(d0[0][0]) < trim])
print 'Trimmed examples:', len(data[0])

Trimmed examples: 86355


In [6]:
data = [map(list, data[0]), map(list, data[1])]

In [ ]:
# debug section below

In [21]:
a = QANet.iterate_minibatches(data, 2)

In [25]:
b = a.next()

In [26]:
questions, contexts, questions_char, contexts_char, bin_feats, question_mask, context_mask, \
                    question_char_mask, context_char_mask, answer_inds =b

In [51]:
questions_char

array([[[1014,  678, 1240,   -1,   -1,   -1,   -1,   -1,   -1],
        [ 665,  303,  765,  918,  385,  918,  753,  303,   -1],
        [1087,  222,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        [1014,  678, 1240,   -1,   -1,   -1,   -1,   -1,   -1],
        [ 765,  303,  753,  540, 1240, 1004,   -1,   -1,   -1],
        [ 678, 1240,  303,  540, 1014,   -1,   -1,   -1,   -1],
        [ 303, 1014,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        [  96, 1087, 1014,  540, 1240,   -1,   -1,   -1,   -1],
        [1004,  303,  611, 1240,   -1,   -1,   -1,   -1,   -1],
        [ 918,  765,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        [ 665, 1240,  765,  918, 1004, 1240,   -1,   -1,   -1],
        [1014, 1087,   -1,   -1,   -1,   -1,   -1,   -1,   -1],
        [ 469,  678,  918,  753,  678,   -1,   -1,   -1,   -1],
        [ 765, 1014,  540, 1247,  753, 1014, 1247,  540, 1240],
        [  76,   -1,   -1,   -1,   -1,   -1,   -1,   -1,   -1]],

       [[ 469,  678,  303, 1014,   -1,

In [37]:
data[1][2]

[[[1014, 678, 1240],
  [665, 303, 765, 918, 385, 918, 753, 303],
  [1087, 222],
  [1014, 678, 1240],
  [765, 303, 753, 540, 1240, 1004],
  [678, 1240, 303, 540, 1014],
  [303, 1014],
  [96, 1087, 1014, 540, 1240],
  [1004, 303, 611, 1240],
  [918, 765],
  [665, 1240, 765, 918, 1004, 1240],
  [1014, 1087],
  [469, 678, 918, 753, 678],
  [765, 1014, 540, 1247, 753, 1014, 1247, 540, 1240],
  [76]],
 [[303,
   540,
   753,
   678,
   918,
   1014,
   1240,
   753,
   1014,
   1247,
   540,
   303,
   385,
   385,
   930],
  [343],
  [1014, 678, 1240],
  [765, 753, 678, 1087, 1087, 385],
  [678, 303, 765],
  [303],
  [753, 303, 1014, 678, 1087, 385, 918, 753],
  [753, 678, 303, 540, 303, 753, 1014, 1240, 540],
  [800],
  [303, 1014, 1087, 75],
  [1014, 678, 1240],
  [611, 303, 918, 96],
  [665, 1247, 918, 385, 1004, 918, 96, 459],
  [419, 765],
  [459, 1087, 385, 1004],
  [1004, 1087, 611, 1240],
  [918, 765],
  [303],
  [459, 1087, 385, 1004, 1240, 96],
  [765, 1014, 303, 1014, 1247, 1240]

In [49]:
data[0][3]

[[[76, 77, 78, 79, 80, 81, 82]],
 [88056, 19554, 67711, 16921, 14138, 87486, 82100, 83077],
 [100780,
  44968,
  67711,
  60695,
  608,
  43315,
  89195,
  32610,
  492,
  40701,
  67711,
  45830,
  54332,
  55792,
  78791,
  78506,
  19554,
  43315,
  51341,
  10820,
  23764,
  67711,
  87146,
  71884,
  492,
  83032,
  94338,
  95628,
  23764,
  67711,
  45830,
  54332,
  49698,
  33470,
  19557,
  44968,
  19554,
  43315,
  85100,
  10820,
  23764,
  99569,
  1485,
  96317,
  37478,
  1485,
  67711,
  78483,
  64851,
  101002,
  14122,
  32833,
  66547,
  77561,
  492,
  19445,
  78406,
  67711,
  45830,
  54332,
  19554,
  67711,
  32756,
  23764,
  67711,
  7991,
  80913,
  492,
  83032,
  71615,
  67711,
  32756,
  19554,
  67711,
  16921,
  44968,
  43315,
  50307,
  87507,
  23764,
  55111,
  49698,
  18814,
  492,
  19557,
  19554,
  43315,
  41529,
  23764,
  67711,
  16921,
  14138,
  72312,
  44968,
  97133,
  39657,
  67711,
  87146,
  71884,
  52733,
  15575,
  78406,
  1

In [31]:
data[0][3][1:]

[[88056, 19554, 67711, 16921, 14138, 87486, 82100, 83077],
 [100780,
  44968,
  67711,
  60695,
  608,
  43315,
  89195,
  32610,
  492,
  40701,
  67711,
  45830,
  54332,
  55792,
  78791,
  78506,
  19554,
  43315,
  51341,
  10820,
  23764,
  67711,
  87146,
  71884,
  492,
  83032,
  94338,
  95628,
  23764,
  67711,
  45830,
  54332,
  49698,
  33470,
  19557,
  44968,
  19554,
  43315,
  85100,
  10820,
  23764,
  99569,
  1485,
  96317,
  37478,
  1485,
  67711,
  78483,
  64851,
  101002,
  14122,
  32833,
  66547,
  77561,
  492,
  19445,
  78406,
  67711,
  45830,
  54332,
  19554,
  67711,
  32756,
  23764,
  67711,
  7991,
  80913,
  492,
  83032,
  71615,
  67711,
  32756,
  19554,
  67711,
  16921,
  44968,
  43315,
  50307,
  87507,
  23764,
  55111,
  49698,
  18814,
  492,
  19557,
  19554,
  43315,
  41529,
  23764,
  67711,
  16921,
  14138,
  72312,
  44968,
  97133,
  39657,
  67711,
  87146,
  71884,
  52733,
  15575,
  78406,
  1880,
  11294,
  70764,
  94338,
 

In [ ]:
# debug end

In [7]:
%%time
data_dev = np.load(squad_path + 'dev_with_training_vocab.pkl')
data_dev_char = np.load(squad_path + 'dev_char_with_training_charlist.pkl')

CPU times: user 8.03 s, sys: 15.5 ms, total: 8.05 s
Wall time: 8.05 s


In [8]:
# Originally contexts are split into sentences, this reverses that.
for i in xrange(len(data_dev)):
    data_dev[i].append(list(chain(*data_dev[i][1][1:])))
    data_dev[i][1] = data_dev[i][1][0]

## QANet

In [23]:
%%time
update_fn = lambda l, p: L.updates.adam(l, p)

qa_net = QANet.QANet(voc_size=voc_size,
                     alphabet_size=alphabet_size,
                     emb_size=300,
                     emb_char_size=100,
                     num_emb_char_filters=200,
                     rec_size=300,
                     emb_init=glove_embs,
                     skip_pred_fns=False,
                     update_fn=update_fn)

Building the model...
Using custom update_fn.
Compiling theano functions:
    train_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 1min 10s, sys: 3.27 s, total: 1min 13s
Wall time: 2min 1s


In [29]:
# z char-embeddings

# 2 ep

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# gdzie się da: glove init, reszta: losowo, trenujemy wszystkie słowa, char-embeddings ma losowy init

qa_net.train_one_epoch(data, 15, log_interval=5)

Done 5 batches in 6.24s	training loss:	2.588375
Done 10 batches in 11.14s	training loss:	2.387342
Done 15 batches in 15.62s	training loss:	2.310226
Done 20 batches in 19.91s	training loss:	2.356700
Done 25 batches in 24.69s	training loss:	2.240458
Done 30 batches in 28.94s	training loss:	2.164079
Done 35 batches in 33.36s	training loss:	2.182367
Done 40 batches in 37.80s	training loss:	2.167793
Done 45 batches in 42.20s	training loss:	2.231234
Done 50 batches in 46.61s	training loss:	2.238087
Done 55 batches in 50.64s	training loss:	2.258154
Done 60 batches in 55.49s	training loss:	2.284326
Done 65 batches in 59.61s	training loss:	2.314155
Done 70 batches in 63.79s	training loss:	2.293630
Done 75 batches in 68.54s	training loss:	2.286849
Done 80 batches in 72.75s	training loss:	2.306152
Done 85 batches in 77.14s	training loss:	2.293647
Done 90 batches in 81.65s	training loss:	2.306131
Done 95 batches in 86.04s	training loss:	2.300315
Done 100 batches in 90.72s	training loss:	2.314458
D

2.453853207685309

In [30]:
qa_net.save_params('trained_models/test_params_char_emb_fixed_2ep')

In [144]:
# qa_net.load_params('test_params_3ep.npz')

In [118]:
# 3 ep

# dane są przycięte do długości 300 (jeśli odpowiedź się nie mieści, to pytanie jest usuwane z danych)
# przycięto około 1400 próbek, usunięto 119

# gdzie się da: glove init, reszta: losowo, trenujemy wszystkie słowa

qa_net.train_one_epoch(data, 20, log_interval=5)

Done 5 batches in 4.76s	training loss:	1.967765
Done 10 batches in 8.98s	training loss:	1.789092
Done 15 batches in 13.79s	training loss:	1.879856
Done 20 batches in 18.41s	training loss:	1.920875
Done 25 batches in 23.06s	training loss:	1.824911
Done 30 batches in 27.20s	training loss:	1.797667
Done 35 batches in 31.60s	training loss:	1.752268
Done 40 batches in 36.46s	training loss:	1.734237
Done 45 batches in 40.70s	training loss:	1.730883
Done 50 batches in 45.06s	training loss:	1.747018
Done 55 batches in 49.39s	training loss:	1.723154
Done 60 batches in 54.32s	training loss:	1.760780
Done 65 batches in 58.55s	training loss:	1.734702
Done 70 batches in 63.20s	training loss:	1.748750
Done 75 batches in 67.94s	training loss:	1.766842
Done 80 batches in 72.43s	training loss:	1.775458
Done 85 batches in 76.76s	training loss:	1.769615
Done 90 batches in 81.45s	training loss:	1.764126
Done 95 batches in 86.00s	training loss:	1.752024
Done 100 batches in 90.34s	training loss:	1.752533
Do

1.7314470210038153

### QANet tests

In [31]:
def predict_spans(data, beam=10, batch_size=10):
    num_examples = len(data[0])
    
    start_probs = qa_net.get_start_probs(data, batch_size)
    best_starts = start_probs.argpartition(-beam, axis=1)[:, -beam:].astype(np.int32)
    
    scores = start_probs[np.arange(num_examples)[:, np.newaxis], best_starts]
    scores = np.tile(scores[:, np.newaxis], (beam, 1)).transpose(0, 2, 1)
    
    best_ends_all = []
    for i in xrange(beam):
        end_probs = qa_net.get_end_probs(data, best_starts[:, i], batch_size)
        best_ends = end_probs.argpartition(-beam, axis=1)[:, -beam:]
        scores[:, i, :] *= end_probs[np.arange(num_examples)[:, np.newaxis], best_ends]
        best_ends_all.append(best_ends)
        
    best_ends_all = np.hstack(best_ends_all)
        
    scores = scores.reshape(num_examples, beam**2)
    best_spans = scores.argmax(axis=1)
    starts = [i / beam for i in best_spans]
    
    starts = best_starts[np.arange(num_examples), starts]
    ends = best_ends_all[np.arange(num_examples), best_spans]
    
    return starts, ends

In [32]:
%%time
predicted_spans = []
batch_size = 10

idx = 0
while idx < len(data_dev):
    spans = predict_spans((data_dev[idx:idx + batch_size], data_dev_char[idx:idx + batch_size]))
    predicted_spans.append(np.vstack(spans))
    idx += batch_size
    if not idx % 100:
        print idx, 'examples'
        
print 'Predictions done'
    
predicted_spans = np.hstack(predicted_spans).T

100 examples
200 examples
300 examples
400 examples
500 examples
600 examples
700 examples
800 examples
900 examples
1000 examples
1100 examples
1200 examples
1300 examples
1400 examples
1500 examples
1600 examples
1700 examples
1800 examples
1900 examples
2000 examples
2100 examples
2200 examples
2300 examples
2400 examples
2500 examples
2600 examples
2700 examples
2800 examples
2900 examples
3000 examples
3100 examples
3200 examples
3300 examples
3400 examples
3500 examples
3600 examples
3700 examples
3800 examples
3900 examples
4000 examples
4100 examples
4200 examples
4300 examples
4400 examples
4500 examples
4600 examples
4700 examples
4800 examples
4900 examples
5000 examples
5100 examples
5200 examples
5300 examples
5400 examples
5500 examples
5600 examples
5700 examples
5800 examples
5900 examples
6000 examples
6100 examples
6200 examples
6300 examples
6400 examples
6500 examples
6600 examples
6700 examples
6800 examples
6900 examples
7000 examples
7100 examples
7200 examples
7

In [33]:
np.savez('evaluate/dev_with_training_vocab_predictions_charemb_fixed_2ep', predicted_spans)

''' 1ep {"f1": 47.51399403406588, "exact_match": 34.24787133396405}'''
''' 3ep {"f1": 47.31910768176034, "exact_match": 32.913907284768214}'''
''' 4ep {"f1": 46.193391429490234, "exact_match": 31.901608325449384}'''

# ??? coś jest nie tak, powinno być lepiej

' 4ep {"f1": 46.193391429490234, "exact_match": 31.901608325449384}'

In [181]:
''' poprzednia wersja:
3ep {"f1": 58.19603067603314, "exact_match": 47.74834437086093}
1ep {"f1": 57.45056892138822, "exact_match": 47.34153263954588}
'''

CPU times: user 5.48 s, sys: 9.56 s, total: 15 s
Wall time: 1.27 s


(2, 20)

## HRED v2

In [4]:
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300,
                lv2_rec_size=300,
                out_emb_size=300,
                emb_init=glove_embs)

Building the model...
Compiling theano functions...
Done


In [5]:
ans = hred_net.get_output(data[:12], 3)
ans[0]

[array([ 0.6484955 ,  0.39728042,  0.29447019,  0.21638568,  0.34087649,
         0.42938796,  0.48248306,  0.45450044,  0.63264418], dtype=float32),
 array([ 0.43808964,  0.31143361,  0.32550314,  0.41264656,  0.40189919,
         0.67630386,  0.55489218,  0.2793248 ,  0.29060629,  0.52473378,
         0.40135068,  0.42928901,  0.24922398,  0.29269892,  0.26922536,
         0.37188664], dtype=float32),
 array([ 0.48105857,  0.35441753,  0.33231139,  0.33165267,  0.25691301,
         0.29037777,  0.38888532,  0.45136821,  0.26165053,  0.35437146,
         0.36461809,  0.29422539,  0.29016727,  0.75232959,  0.50326639,
         0.4879126 ,  0.19824371,  0.41686958,  0.44867724,  0.63904244,
         0.59564072,  0.3762525 ,  0.55565   ,  0.20368919,  0.45814279,
         0.13423204,  0.25729677,  0.28506696,  0.130806  ,  0.43678874], dtype=float32),
 array([ 0.36110905,  0.18258747,  0.213524  ,  0.28226432,  0.41397128,
         0.32674885,  0.29788589,  0.61358768,  0.34168947,  0.22

In [12]:
list(chain(*ans[8]))[46]

0.05232447

In [11]:
get_w(list(chain(*data[8][1][1:]))[45])

u'september'

In [10]:
list(reversed(np.array(list(chain(*ans[8]))).argsort()))

[46,
 23,
 45,
 74,
 7,
 19,
 49,
 8,
 6,
 126,
 10,
 24,
 28,
 80,
 9,
 29,
 54,
 43,
 31,
 37,
 4,
 42,
 123,
 82,
 11,
 30,
 111,
 94,
 16,
 35,
 81,
 15,
 50,
 148,
 62,
 78,
 53,
 91,
 204,
 33,
 13,
 12,
 105,
 21,
 20,
 155,
 152,
 122,
 36,
 158,
 125,
 203,
 217,
 5,
 22,
 84,
 14,
 87,
 27,
 34,
 3,
 85,
 107,
 222,
 186,
 79,
 25,
 110,
 128,
 56,
 175,
 60,
 127,
 131,
 52,
 117,
 63,
 149,
 112,
 47,
 26,
 66,
 55,
 2,
 32,
 86,
 197,
 171,
 151,
 61,
 108,
 83,
 137,
 119,
 124,
 48,
 67,
 166,
 201,
 134,
 239,
 1,
 172,
 44,
 236,
 90,
 233,
 106,
 77,
 240,
 104,
 176,
 71,
 141,
 75,
 0,
 227,
 142,
 101,
 109,
 198,
 147,
 202,
 154,
 73,
 207,
 18,
 235,
 226,
 93,
 65,
 59,
 150,
 120,
 194,
 113,
 195,
 205,
 121,
 118,
 153,
 57,
 232,
 39,
 17,
 216,
 132,
 41,
 140,
 247,
 116,
 178,
 64,
 97,
 179,
 100,
 58,
 224,
 76,
 170,
 70,
 69,
 51,
 243,
 103,
 133,
 169,
 136,
 218,
 237,
 238,
 143,
 72,
 174,
 167,
 102,
 225,
 177,
 200,
 89,
 181,
 114,
 146,
 20

In [ ]:
# Zwykły trening nie robił nic: wartości zwracane przez sieć dla zbioru uczącego nie miały żadnego sensu.
# Największe prawdopodobieństwa były prawie niezależne od pytania, miało ono mały wpływ na to, co się dzieje.
# Dla kosztu weighted_bin_ce mocno skrzywionego w stronę sytuacji t=1 wyszło podobnie.
#
# Następny krok: użyć zanurzeń GloVe, dodać cechy z https://arxiv.org/abs/1703.04816
#
# UPDATE:
# Obie cechy podobieństwa kontekstu do pytania dodane.
# W glove.6B jest około 70% słów ze zbioru uczącego SQuAD.
# W tej chwili te wektory biorę z glove.6B.300d, a pozostałym daję losowy init, 
# po czym wszystkie zanurzenia są dalej uczone.

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # weighted cost, random init

Done 20 batches in 7.96s	training loss:	0.813393
Done 40 batches in 16.89s	training loss:	0.618499
Done 60 batches in 25.51s	training loss:	0.537964
Done 80 batches in 33.72s	training loss:	0.476587
Done 100 batches in 41.79s	training loss:	0.436442
Done 120 batches in 48.56s	training loss:	0.419979
Done 140 batches in 57.10s	training loss:	0.409618
Done 160 batches in 65.07s	training loss:	0.408088
Done 180 batches in 74.05s	training loss:	0.397444
Done 200 batches in 81.68s	training loss:	0.401948
Done 220 batches in 90.27s	training loss:	0.401396
Done 240 batches in 98.95s	training loss:	0.410639
Done 260 batches in 107.43s	training loss:	0.418680
Done 280 batches in 115.18s	training loss:	0.427639
Done 300 batches in 123.33s	training loss:	0.426674
Done 320 batches in 131.00s	training loss:	0.422292
Done 340 batches in 139.84s	training loss:	0.424456
Done 360 batches in 148.91s	training loss:	0.415105
Done 380 batches in 156.67s	training loss:	0.416838
Done 400 batches in 163.90s	t

KeyboardInterrupt: 

In [5]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with random init

Done 20 batches in 2.25s	training loss:	0.228349
Done 40 batches in 5.03s	training loss:	0.154742
Done 60 batches in 7.65s	training loss:	0.126147
Done 80 batches in 10.05s	training loss:	0.106807
Done 100 batches in 12.52s	training loss:	0.095184
Done 120 batches in 14.64s	training loss:	0.090502
Done 140 batches in 17.25s	training loss:	0.087078
Done 160 batches in 19.51s	training loss:	0.085557
Done 180 batches in 22.31s	training loss:	0.082334
Done 200 batches in 24.40s	training loss:	0.082256
Done 220 batches in 26.96s	training loss:	0.081215
Done 240 batches in 29.60s	training loss:	0.082376
Done 260 batches in 32.51s	training loss:	0.083784
Done 280 batches in 34.97s	training loss:	0.084956
Done 300 batches in 37.36s	training loss:	0.084330
Done 320 batches in 39.52s	training loss:	0.083391
Done 340 batches in 42.24s	training loss:	0.083563
Done 360 batches in 44.99s	training loss:	0.081207
Done 380 batches in 47.14s	training loss:	0.081512
Done 400 batches in 49.09s	training lo

KeyboardInterrupt: 

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with glove init

Done 20 batches in 2.66s	training loss:	0.247073
Done 40 batches in 5.97s	training loss:	0.187469
Done 60 batches in 9.10s	training loss:	0.148785
Done 80 batches in 11.96s	training loss:	0.123427
Done 100 batches in 14.91s	training loss:	0.108109
Done 120 batches in 17.43s	training loss:	0.102956
Done 140 batches in 20.55s	training loss:	0.097911
Done 160 batches in 23.25s	training loss:	0.095246
Done 180 batches in 26.59s	training loss:	0.090887
Done 200 batches in 30.07s	training loss:	0.090136
Done 220 batches in 33.13s	training loss:	0.088647
Done 240 batches in 36.28s	training loss:	0.089258
Done 260 batches in 39.74s	training loss:	0.091964
Done 280 batches in 42.68s	training loss:	0.093008
Done 300 batches in 45.52s	training loss:	0.091937
Done 320 batches in 48.11s	training loss:	0.090498
Done 340 batches in 51.34s	training loss:	0.090224
Done 360 batches in 54.62s	training loss:	0.087337
Done 380 batches in 57.19s	training loss:	0.087323
Done 400 batches in 59.53s	training lo

KeyboardInterrupt: 

In [ ]:
hred_net.save_params()